# Fake 2d Detector Image Generator

``fake_2d_detector_img_generator`` in the module ``beamlime.resources.images.generators``

creates a number of frames(2d arrays) from the ``seed_img`` and yield one at a time.

We could use real physics ``simulation`` to generate a fake data, but that is too expensive,

so it will only create predictable fake-data in a simple way.

Here it shows how the ``seed_img`` is manipulated and how each frame looks like.

In [ ]:
import scipp as sc
import plopp as pp
import numpy as np
from matplotlib import pyplot as plt
from beamlime.resources.images import load_icon_img
from beamlime.resources.images.generators import fake_2d_detector_img_generator

%matplotlib widget

## Original Image

We are going to use our lime icon image as a seed image

as if we are taking a picture of imbaglio-carved lime on a metal plate.

The original icon image looks like below.

In [ ]:
from PIL import Image
from PIL.ImageOps import flip
original_img = Image.fromarray(np.uint8(load_icon_img()))
plt.imshow(Image.fromarray(np.uint8(original_img)))

## Seed Image

And then we will reduce the image size to avoid too much computing.

We also flip this since the image (0, 0) is top-left and histogram plot (0, 0) is top-bottom

In [ ]:
detector_size = (64, 64)
resized_img = original_img.resize(detector_size)
seed_img = np.asarray(flip(resized_img), dtype=np.float64)

## Fake Images

Now we will generate the images and accumulate them.


### Without Noise

First, we will set ``noise_range`` to 0, to see the clean view.

In [ ]:
base = sc.DataArray(data=sc.zeros(sizes={'y': detector_size[1], 'x': detector_size[0]}),
                    coords={'x': sc.linspace('x', 0, detector_size[0]+1, detector_size[0]+1, unit='mm'),
                            'y': sc.linspace('y', 0, detector_size[1]+1, detector_size[1]+1, unit='mm')})

generator = iter(fake_2d_detector_img_generator(seed_img, num_frame=128, min_intensity=0.5,
                                                signal_mu=0.5, signal_err=0.3,
                                                noise_mu=0, noise_err=0))

for isimg, simg in enumerate(generator):
    base.values *= (isimg/(isimg+1))
    base.values += simg/(isimg+1)

base.plot(title="Without Noise")

### With Noise

And now we can set the ``noise_range`` to 0.55 then the images look more dirty.

The lime is almost hidden in the noise.

It is for the dummy data reduction, which will filter the pixels by threshold.

After the data reduction, you will be able to see the lime.

In [ ]:
plt.close()

In [ ]:
base = sc.DataArray(data=sc.zeros(sizes={'y': detector_size[1], 'x': detector_size[0]}),
                    coords={'x': sc.linspace('x', 0, detector_size[0]+1, detector_size[0]+1, unit='mm'),
                            'y': sc.linspace('y', 0, detector_size[1]+1, detector_size[1]+1, unit='mm')})

generator = iter(fake_2d_detector_img_generator(seed_img, num_frame=128, min_intensity=0.5,
                                                signal_mu=0.3, signal_err=0.3,
                                                noise_mu=1, noise_err=0.3))
for isimg, simg in enumerate(generator):
    base.values *= (isimg/(isimg+1))
    base.values += simg/(isimg+1)

base.plot(title="With Noise (mu=1, std=0.3)")

### Single Frame

Each frame looks like below.

They are all randomly split so every time you call the generator, it will create a different set of images.

Note that it creates all the frames at once when it's first called, and yield one at a time.

So don't use too high number for the frames.

If the ``num_frame`` is higher, each frame will contain less valid pixel above threshold.

In [ ]:
fig, axs = plt.subplots(3, 6, figsize=(12,6))
fig.suptitle('Detector Image Frames To Be Fed - First 18')

generator = iter(fake_2d_detector_img_generator(seed_img, num_frame=128, min_intensity=0.5,
                                                signal_mu=0.5, signal_err=0.3,
                                                noise_mu=0.9, noise_err=0.3))
for iframe, simg in enumerate(generator):
    frame = sc.DataArray(data=sc.array(dims=['x', 'y'], values=simg),
                        coords={'x': sc.linspace('x', 0, detector_size[0]+1, detector_size[0]+1, unit='mm'),
                                'y': sc.linspace('y', 0, detector_size[1]+1, detector_size[1]+1, unit='mm')})
    if iframe < 18:
        ax_x = iframe % 6
        ax_y = iframe // 6
        ax = axs[ax_y][ax_x]
        frame.plot(ax=ax).cax.set(ylabel=None, yticks=[])
        ax.set(xlabel=None, ylabel=None, xticks=[], yticks=[])

# Fake Data Stream

``Fake2dDetectorImageFeeder`` in the ``beamlime.offline.data_feeder`` uses this generator.

The data feeder will wrap this image into a dictionary and send it to the stream.

It is the example of the dictionary it will return.

In [ ]:
first_frame = next(fake_2d_detector_img_generator(seed_img))
fake_data = {
                "sample_id": "typical-lime-intaglio-0",
                "timestamp": 0,
                "detector-data": {
                    "detector-id": "unknown-2d-detector",
                    "image": frame,
                },
            }


fake_data['detector-data'] = sc.DataGroup(fake_data['detector-data'])
sc.DataGroup(fake_data)  ## wrapping dictionary with scipp.DataGroup for formatting.